In [1]:
### STEP 1 --讀檔及載入套件--
# 做完特徵工程前的所有準備 (與前範例相同)

import pandas as pd
import numpy as np
import copy, time
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso

data_path = '/Users/maylin/Documents/Data/'
df_train = pd.read_csv(data_path + 'house_train.csv')
df_test = pd.read_csv(data_path + 'house_test.csv')


In [2]:
df_test.shape

(1459, 80)

In [3]:
df_train.shape

(1460, 81)

In [6]:
### STEP 2 --資料處理 產生初步的 TRAIN DATA 及 TEST DATA --
## train 先排除Y，將Train + Test 再來處理
train_Y = df_train['SalePrice']
ids = df_test['Id']
df_train = df_train.drop(['Id', 'SalePrice'] , axis=1) #去除name,poi 先去除index和Y
df_test = df_test.drop(['Id'] , axis=1)
test_X = df_test
df = pd.concat([df_train,df_test]) 
df.shape

(2919, 79)

In [7]:
# 檢查 DataFrame 空缺值的狀態
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(20))
na_check(df)

,Missing Ratio
PoolQC,99.657417
MiscFeature,96.402878
Alley,93.216855
Fence,80.438506
FireplaceQu,48.646797
LotFrontage,16.649538
GarageFinish,5.447071
GarageYrBlt,5.447071
GarageQual,5.447071
GarageCond,5.447071


In [8]:
# == practice == 把遺缺值七成以上的特徵排除 直接看下面的空值比例高拿掉 這裡拿掉前4個
df = df.drop(['PoolQC', 'MiscFeature','Alley','Fence'] , axis=1)
df.shape

(2919, 75)

In [11]:
df.dtypes.value_counts()

object     39
int64      25
float64    11
dtype: int64

In [38]:
###here!!!!!!!!
## 離群值處理 outlier 
# 將 1stFlrSF 限制在你覺得適合的範圍內, 調整離群值
##"""
##Your Code Here
##"""

##df['1stFlrSF'] = df['1stFlrSF'][:train_num].clip(600, 1750)
##sns.regplot(x = df['1stFlrSF'][:train_num], y=train_Y)
##plt.show()

In [39]:
## 補空值的方法 (下面三種可換)
df=df.fillna(0)
#df = df.fillna(-1)
#df = df.fillna(df.mean())

In [12]:
# 處理數值型資料
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中 
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
# df = df[num_features]
# train_num = train_Y.shape[0]
# test_num = test_X.shape[0]


36 Numeric Features : ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']



In [40]:
## 最大最小化 或 標準化處理
## https://www.itread01.com/content/1547461823.html
## 如果資料真的有outlier，要做outlier完才做最大最小化 或 標準化處理，效果才會比較好

MMEncoder = MinMaxScaler()

## X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
## X_scaled = X_std * (max - min) + min

STDEncoder = StandardScaler()

##
##
df.head()

,bonus,deferred_income,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,long_term_incentive,other,restricted_stock,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,-3504386.0,19794175.0,46950.0,18.0,42.0,4.0,1617011.0,174839.0,2748364.0,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,0.0,19250000.0,29336.0,108.0,88.0,30.0,1920000.0,22122.0,6843672.0,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,-4167.0,1624396.0,22884.0,39.0,13.0,14.0,0.0,1573324.0,869220.0,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,0.0,0.0,118134.0,0.0,0.0,0.0,602671.0,907502.0,985032.0,224305.0,0.0,0.0,2652612.0,985032.0
4,1250000.0,-262500.0,0.0,35818.0,144.0,199.0,25.0,375304.0,486.0,126027.0,240189.0,2188.0,2598.0,1639297.0,126027.0


In [41]:
## 特徵工程 (1) 相關係數 (2) Lasso (3)Ridge  那些特徵比較有相關？！！！！ 決定性的特徵！！！

## PRACTICE 使用 CORR 相關係數，刪除特徵值 參考D40

## 觀察相關係數,列出目標 (TARGET) 與所有欄位之間相關係數，數值最大以及最小各 15 個
df_corr = df_train_corr.corr()
#print(df_corr.to_xarray()) 
print(df_corr['poi'].sort_values())

# max_corr = df_corr[1:12]
# min_corr = df_corr[-10:]
#print(max_corr)
##print(min_corr)

# 參考D30 篩選相關係數1

corr_features = list(df_corr[(df_corr['poi']>0.1) | (df_corr['poi']<-0.1)].index)
corr_features.remove('poi')

# 把遺缺值七成以上的特徵排除
##nan_features = ('loan_advances','director_fees','restricted_stock_deferred','deferral_payments')
##for i in nan_features:
##    corr_features.remove(i)

 
print(len(corr_features))
print(corr_features)
corr_features.pop(0)
print(type(corr_features))

L1_list = corr_features

###-not finish- df = df[corr_features]
###-not finish- train_X = MMEncoder.fit_transform(df[corr_features])
###-not finish- cross_val_score(estimator, train_X, train_Y, cv=5).mean()


deferred_income           -0.224654
from_messages             -0.126760
from_this_person_to_poi   -0.096864
to_messages               -0.023047
shared_receipt_with_poi    0.120467
from_poi_to_this_person    0.127932
expenses                   0.136386
other                      0.225619
long_term_incentive        0.227805
bonus                      0.242170
restricted_stock           0.261102
total_payments             0.271335
salary                     0.371917
total_stock_value          0.443098
exercised_stock_options    0.591972
poi                        1.000000
Name: poi, dtype: float64
13
['bonus', 'deferred_income', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'long_term_incentive', 'other', 'restricted_stock', 'salary', 'shared_receipt_with_poi', 'total_payments', 'total_stock_value']
<class 'list'>


In [42]:
## PRACTICE 使用LASSO 檢查共線性，刪除特徵值 參考D40
from sklearn.linear_model import Lasso
# 切分訓練集/測試集
##x_train, x_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, test_size=0.2, random_state=4)
train_X=df[:train_num]
test_X=df[:test_num]
# 建立一個線性回歸模型
lasso = Lasso(alpha=10000) #alpha 越大縮越快

# 將訓練資料丟進去模型訓練
lasso.fit(train_X, train_Y)

# 將測試資料丟進模型得到預測結果
y_pred = lasso.predict(test_X)

# 檢查相關性
lasso.coef_

from itertools import compress
L1_mask = list((lasso.coef_>0) | (lasso.coef_<0)) # 取不等於0的
print(len(L1_mask),type(L1_mask),L1_mask)
L1_list = list(compress(list(df), list(L1_mask)))  
L1_list.pop(0)

print(len(L1_list),type(L1_list),L1_list) #L1_List最後送Model
#最後挑出9個特徵送Model

15 <class 'list'> [True, True, True, False, False, False, False, False, False, False, False, False, False, True, True]
4 <class 'list'> ['deferred_income', 'exercised_stock_options', 'total_payments', 'total_stock_value']


In [43]:
### PRACTICE 確認最後選完的特徵 ，確認 train_X test_X 是否適合  ###

# 開始切分資料，特徵選完進模型
train_X=df[:train_num]
test_X=df[:test_num]
train_X =  MMEncoder.fit_transform(train_X[L1_list])
test_X =  MMEncoder.fit_transform(test_X[L1_list])

print(df.shape)
print(train_num)
print(test_X.shape)
print(train_X.shape)
print(train_Y.shape)

### 標準化完成 ###

(146, 15)
113
(33, 4)
(113, 4)
(113,)


In [44]:
print(train_X)


[[0.00000000e+00 5.76276747e-01 4.87689271e-03 4.59505908e-01]
 [1.00000000e+00 5.60433935e-01 8.38425392e-02 5.31750703e-01]
 [9.98810919e-01 4.72917736e-02 1.93500290e-02 5.16275414e-02]
 [1.00000000e+00 0.00000000e+00 2.56143038e-02 2.09366770e-02]
 [9.25093868e-01 0.00000000e+00 1.58294735e-02 3.46094739e-03]
 [1.00000000e+00 8.95706302e-01 8.79617440e-04 6.26806563e-01]
 [1.00000000e+00 2.41890623e-01 3.47914948e-03 2.42682376e-01]
 [9.58890944e-01 0.00000000e+00 1.43911450e-02 1.51021772e-02]
 [9.14392992e-01 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.12007598e-02 1.22855016e-02 1.67358941e-02]
 [1.00000000e+00 1.95565823e-02 1.53285455e-02 3.99589284e-02]
 [9.71464331e-01 4.76074216e-02 1.06131923e-02 3.99351664e-02]
 [6.04479929e-01 0.00000000e+00 2.34075690e-02 3.74028279e-02]
 [1.00000000e+00 1.27084581e-02 2.07317236e-02 1.87512877e-02]
 [1.20800334e-01 5.03529074e-02 4.33029255e-02 3.60830823e-02]
 [1.00000000e+00 2.22809026e-02 1.94557650e-02 2.031255

In [45]:
#### PRACTICE 參數最佳化 ####
## 模型參數最佳化  D47
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# 設定要訓練的超參數組合
n_estimators = [50, 100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

clf = GradientBoostingRegressor(random_state=7)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(train_X, train_Y)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

grid_result.best_params_

# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Accuracy: -0.155146 using {'max_depth': 1, 'n_estimators': 50}


[Parallel(n_jobs=-1)]: Done  29 out of  36 | elapsed:    2.1s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    2.3s finished


In [46]:
### PRACTICE 確認最後選完的特徵 ，確認 train_X test_X 是否適合  ###

# 使用三種模型 : 邏輯斯迴歸 / 梯度提升機 / 隨機森林, 參數使用 Random Search 尋找
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
lr = LogisticRegression()
gdbt = GradientBoostingClassifier()
rf = RandomForestClassifier()

In [47]:
# 邏輯斯迴歸預測檔 (結果有部分隨機, 請以 Kaggle 計算的得分為準, 以下模型同理)
lr.fit(train_X, train_Y)
prediction = lr.predict_proba(test_X)[:,1]
print(lr.coef_)
print(lr.intercept_)
print(prediction)

[[-1.3153751   1.30164915  0.38013788  1.11029179]]
[-0.93439372]
[0.58108211 0.28936356 0.10688942 0.09816675 0.10504614 0.40420537
 0.15910583 0.10206953 0.65811726 0.09859646 0.10201101 0.10882028
 0.15708951 0.09925725 0.27444486 0.10048974 0.09658885 0.09609813
 0.09603292 0.09692761 0.12252079 0.13192316 0.10050012 0.10130283
 0.09960734 0.10533271 0.09782872 0.09593762 0.09547078 0.10051627
 0.09588595 0.1140279  0.10433424]


In [48]:
# 邏輯斯迴歸預測檔 (結果有部分隨機, 請以 Kaggle 計算的得分為準, 以下模型同理)
lr.fit(train_X, train_Y)
lr_pred = lr.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': lr_pred})
#sub['poi'] = sub['poi'].map(lambda x:1 if x>0.5 else 0) 
# 上面，變成機率方式

sub.to_csv('marathon_lr_TEST.csv', index=False) 

In [49]:
#### ** not finish **  檢查AUC (需要有正確的test_Y)
####
#### acc = metrics.accuracy_score(y_test, y_pred)
#### print("Accuracy: ", acc)
#### print("Feature importance: ", clf_bestparam.feature_importances_)

In [50]:
# 梯度提升機預測檔 
gdbt.fit(train_X, train_Y)
gdbt_pred = gdbt.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': gdbt_pred})
#sub['poi'] = sub['poi'].map(lambda x:1 if x>0.5 else 0) 
sub.to_csv('marathon_gdbt_TEST.csv', index=False)

In [51]:
# 隨機森林預測檔
rf.fit(train_X, train_Y)
rf_pred = rf.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': rf_pred})
#sub['poi'] = sub['poi'].map(lambda x:1 if x>0.5 else 0) 
sub.to_csv('marathon_rf_TEST.csv', index=False)

In [52]:
### ==== PRACTICE ENSEMBLE ==== Stacking & blending ### 

In [53]:
from mlxtend.classifier import StackingClassifier

meta_estimator = GradientBoostingClassifier(tol=100, subsample=0.70, n_estimators=50, 
                                           max_features='sqrt', max_depth=4, learning_rate=0.3)
stacking = StackingClassifier(classifiers=[lr,gdbt,rf],meta_classifier=meta_estimator)
stacking.fit(train_X, train_Y)
#stacking_pred = stacking.predict(test_X)
stacking_pred = stacking.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': stacking_pred})
sub.to_csv('marathon_stacking_TEST.csv', index=False)

In [54]:
blending_pred = lr_pred*0.25  + gdbt_pred*0.3 + rf_pred*0.45  #這兒可以調整參數  #看三種成績來取權重
sub = pd.DataFrame({'name': ids, 'poi': blending_pred}) # name當key，才可以送測
sub.to_csv('marathon_blending_TEST.csv', index=False)